Testing to read the zarr data produced by Aasmund

In [1]:
import xarray as xr
import numpy as np
#import requests
#from bs4 import BeautifulSoup
import cartopy.crs as ccrs
import pyproj
import pyresample
#import dask.array as da
#from dask.distributed import Client
#from dask_jobqueue import SLURMCluster
#from numcodecs.blosc import Blosc


In [2]:
carra_analysis = xr.open_zarr(f"/scratch/fab0/Projects/cerise/carra_snow_data/ana_v2.zarr")

In [ ]:
carra_analysis.to_netcdf("analysis_zarr_test.nc")